### Preliminaries

Before we get into today's content on pivoting, let's spend a few minutes comparing our original Gaussian Elimination procedure to our new LU decomposition and tridiagonal solvers.

Start by using the code cell below to import the `{numpy}` and `{time}` modules. We'll use `time.time()` to estimate execution time for our procedures.

Now paste the routines we've developed so far (`GaussElim()`, `DoolittleLUsolver()`, and `DoolittleLUdecomp3solver()`, along with their necessary helper functions) into this notebook to make them available to you here.

**Example:** Solve the tridiagonal system $\left[\begin{array}{rrr|r} 2 & -1 & 0 & 1\\ -1 & 2 & -1 & 0\\ 0 & -1 & 1 & 0\end{array}\right]$ using our three techniques. Verify that you get the same solutions with each. Time the methods to determine which ones are faster or slower and discuss why this might be the case.

> **Note:** To time your methods you can use `tic = time.time()` and `toc = time.time()` and then calculate elapsed time as `toc - tic`.

**Example:** Repeat the process, but solve over several constant vectors $\vec{b_1} = \begin{bmatrix} 1\\ 0\\ 0\end{bmatrix}$, $\vec{b_2} = \begin{bmatrix} -2\\ 3\\ 5\end{bmatrix}$, $\vec{b_3} = \begin{bmatrix} 0\\ 4\\ -1\end{bmatrix}$, and $\vec{b_4} = \begin{bmatrix} 0\\ 0\\ -3\end{bmatrix}$. For each of our routines, what is the best way to utilize it to handle this scenario? Is one routine preferred over others this time?

# Day 8: Pivoting

Because of rounding errors, the order of the rows in a coefficient matrix can greatly impact performance and accuracy of results produced by a numerical solver. For example, consider the system

$$\left\{\begin{array}{lcl} 2x_1 - x_2 & = & 1\\
-x_1 + 2x_2 - x_3 & = & 0\\
-x_2 + x_3 & = & 0\end{array}\right.$$

which has the augmented coefficient matrix

$$\left[\begin{array}{ccc|c} 2 & -1 & 0 & 1\\
-1 & 2 & -1 & 0\\
0 & -1 & 1 & 0\end{array}\right]$$

This augmented coefficient matrix has the rows in the "*right order*". We can begin from the top left entry and row-reduce using the *Gaussian Elimination* strategy. However, what would happen if the first and third equations were switched? We'd have an augmented coefficient matrix like the one below.

$$\left[\begin{array}{ccc|c} 0 & -1 & 1 & 0\\
-1 & 2 & -1 & 0\\
2 & -1 & 0 & 1\end{array}\right]$$

Now, *we* would have no problem seeing that having a $0$ in the top left position will not allow us to reduce to a matrix in upper triangular form -- *we* would switch the first row with one of the other rows prior to row reducing. A computer, at least one following the routines we've written so far, would be stumped by this!

Sometimes it is imperative that we reorder the rows in a system during the *elimination phase* -- this is called *row pivoting*. In general, we'll want to do this when the *pivot element* is $0$ or when it is very small compared to the other elements in the pivot row. Consider the following augmented coefficient matrix with $\varepsilon$ representing a very small number.

\begin{align*}\left[\begin{array}{ccc|c} \varepsilon & -1 & 1 & 0\\
-1 & 2 & -1 & 0\\
2 & -1 & 0 & 1\end{array}\right] &\stackrel{R_2 \leftarrow R_2 - (-1/\varepsilon)R_1}{\to}\left[\begin{array}{ccc|c} \varepsilon & -1 & 1  & 0\\
0 & 2 - (1/\varepsilon) & -1 + (1/\varepsilon) & 0\\
2 & -1 & 0 & 1\end{array}\right]\\
&\stackrel{R_3 \leftarrow R_3 - (2/\varepsilon)R_1}{\to}\left[\begin{array}{ccc|c} \varepsilon & -1 & 1  & 0\\
0 & 2 - (1/\varepsilon) & -1 + (1/\varepsilon) & 0\\
0 & -1 + (2/\varepsilon) & -2/\varepsilon & 1\end{array}\right]
\end{align*}

Notice though that since $\varepsilon$ is very small, $1/\varepsilon$ is very large. We may be in danger of entering roundoff error territory. The resulting augmented coefficient may be stored as

$$\left[\begin{array}{ccc|c} \varepsilon & -1 & 1  & 0\\
0 & - 1/\varepsilon & 1/\varepsilon & 0\\
0 & 2/\varepsilon & -2/\varepsilon & 1\end{array}\right]$$

In this case, $R_2$ and $R_3$ contradict eachother and we would identify *no solutions*, but the solution should exist and should be very near $\vec{x} = \left[\begin{array}{c} 1\\ 1\\ 1\end{array}\right]$ (the solution to the original system). Beginning the procedure with a *row pivot* (row swap) avoids this problem altogether. The question now is, how do we identify when a *row pivot* should be performed.


## Diagonal Dominance

An $n\times n$ matrix $A$ is said to be *diagonally dominant* if each diagonal element is greater than the sum of the absolute values of the other elements in its row. That is, if

$$\left|A_{ii}\right| > \sum_{\substack{j = 1\\ j\neq i}}^{n}{\left|A_{ij}\right|}$$

For example, the matrix $\left[\begin{array}{ccc} -2 & 4 & -1\\
1 & -1 & 3\\
4 & -2 & 1\end{array}\right]$ is not diagonally dominant, but two *row pivots* and we obtain $\left[\begin{array}{ccc} 4 & -2 & 1\\
-2 & 4 & -1\\
1 & -1 & 3
\end{array}\right]$, which is diagonally dominant.

If $A$ is diagonally dominant, then a numerical solution for $A\vec{x} = \vec{b}$ *does not* benefit by row pivoting. Because of this, we'll implement a strategy that reorders rows of $A$ to achieve *near*-diagonal-dominance.

## An Example for Illustration

As usual when we are about to introduce a new algorithm, we'll use an example to help illuminate the steps involved. We'll use the example below.

**Example:** Use Gaussian Elimination with Scaled Row pivoting to solve the system $A\vec{x} = \vec{b}$ where $A = \left[\begin{array}{ccc} 2 & -2 & 6\\
-2 & 4 & 3\\
-1 & 8 & 4\end{array}\right]$ and $\vec{b} = \left[\begin{array}{c} 16\\ 0\\ -1\end{array}\right]$.

> *Solution.*

## Gaussian Elimination with Scaled Row Pivoting

Consider an approach to solving the system $A\vec{x} = \vec{b}$ using *Gaussian Elimination* with row-pivoting. That is, we row-pivot $A$ during elimination so that the *pivot element* is as large as possible with respect to the other elements in the pivot row. In order to do this, let's define an array $s$ as follows:

$$s_i = \max_{j}{\left|A_{ij}\right|}~~\text{for}~~i,j\in [n]$$

We call $s_i$ the *scale factor* for $R_i$, and it contains the absolute value of the largest element in the $i^{th}$ row of $A$. We can write a simple routine to populate $s$.

```
#populate scale-factor array
n = ___
s = np.zeros(___)
for i in range(n):
  s[i] = ___(___(A[___, :]))
```

Using $s$, we can compute the relative size of any matrix element to the largest element in its row. That is, $\displaystyle{r_{ij} = \frac{\left|A_{ij}\right|}{s_i}}$.

Suppose we are at the stage of the *elimination phase* where the $k^{th}$ row has become the pivot row. The augmented coefficient matrix is as seen below.

$$\left[\begin{array}{ccccccc|c} A_{11} & A_{12} & A_{13} & \cdots & \cdots & \cdots & A_{1n} & b_1\\
0 & A_{22} & A_{23} & \cdots & \cdots & \cdots & A_{2n} & b_2\\
0 & 0 & A_{33} & \cdots & \cdots & \cdots & A_{3n} & b_3\\
\vdots & \vdots & \vdots & \ddots & \cdots & \vdots & \vdots\\ \hline
0 & 0 & \cdots & 0 & A_{kk} & \cdots & A_{kn} & b_k\\
\vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & \cdots & 0 & A_{nk} & \cdots & A_{nn} & b_n\\
\end{array}\right]$$

Unlike our previous techniques, we don't immediately use $A_{kk}$ as the next pivot element. Instead, we search the rows below $R_k$ for a "better" pivot in column $k$. The best choice is the entry with the largest relative size -- that is, $A_{pk}$ such that $\displaystyle{r_{pk} = \max_{j}{(r_{jk})}}$ for $j\geq k$. If $p \neq k$, then we execute a row exchange both in $A$ and also exchange the corresponding elements in $s$. This can be done as follows:

```
#row-pivot
for k in range(0, n-1):
  p = argmax(abs(A[k:n, k])/s[k:n]) + k
  if abs(A[p, k]) < tol:
    print("Matrix is Singular")
    return None

  if p != k:
    ___
    ___
    ___
```

The algorithm above makes use of a `swapRows()` function, which we'll also write when we build our `gaussPivot()` function for Gaussian Elimination with Scaled Row Pivoting below. Having a `swapRows()` function makes our code easier to read and follow.

In [ ]:
def swapRows(v, i, j):
  if len(v.shape) == 1:
    v[i], v[j] = v[j], v[i]
  else:
    v[[i, j], :] = v[[j, i], :]

def scaleFactors(A):
  n = ___
  s = np.zeros(___)
  for i in range(n):
    s[i] = ___(np.___(A[___, :]))

  return s

def gaussPivot(A, b, tol = 1.0e-12):
  n = ___
  s = scaleFactors(___)

  for k in range(0, n-1):
    p = np.argmax(np.abs(A[k:n, k])/s[k:n]) + k
    if(abs(A[p, k]) < tol):
      print("Matrix is Singular")
      return None

    #Row Pivot if necessary
    if p != k:
      ___
      ___
      ___

    #Elimination
    for i in range(k+1, n):
      if A[i, k] != 0.0:
        lam = A[i, k]/A[k, k]
        A[i, (k+1):n] = A[i, (k+1):n] - lam*A[k, (k+1):n]
        b[i] = b[i] - lam*b[k]

  if abs(A[n-1, n-1]) < tol:
    print("Matrix is Singular")
    return None

  #back substitution
  b[n-1] = b[n-1]/A[n-1, n-1]
  for k in range(n-2, -1, -1):
    b[k] = (b[k] - np.dot(A[k, (k+1):n], b[(k+1):n]))/A[k, k]

  return b

Let's now use the function(s) we've just built to quickly solve our example problems!

**Example:** Recall that our original `gaussElim()` procedure couldn't correctly solve the system below because of the order of the rows. Show that our new `gaussPivot()` method does solve the system correctly.

$$\left[\begin{array}{ccc|c} 0 & -1 & 1 & 0\\
-1 & 2 & -1 & 0\\
2 & -1 & 0 & 1\end{array}\right]$$

**Example:** Use Gaussian Elimination with Scaled Row pivoting to solve the system $A\vec{x} = \vec{b}$ where $A = \left[\begin{array}{ccc} 2 & -2 & 6\\
-2 & 4 & 3\\
-1 & 8 & 4\end{array}\right]$ and $\vec{b} = \left[\begin{array}{c} 16\\ 0\\ -1\end{array}\right]$.

## When to Pivot

There are some drawbacks to pivoting. In particular,

+ those `rowSwap()` operations can be *expensive* (run time), and
+ swapping rows destroys matrix symmetry and any banded structure

Fortunately, in many applications where banded or symmetric matrices appear, those matrices are nearly diagonally dominant and so there is not often a benefit to pivoting anyway. While not hard *rules*, banded matrices, symmetric matrices, and positive definite matrices seldom benefit from row pivoting. Note that it is, however, possible to construct banded, symmetric, or positive definite matrices that *do* benefit from pivoting -- they just don't stem from real engineering problems.

***

## Summary

In this notebook we saw how to use scaled row pivoting to ensure that we don't end up in a situation where a *pivot element* is $0$ or nearly $0$. If a *pivot element* is $0$, our *Gaussian Elimination* procedure will fail, and we'll obtain a misleading "solution" since the backward substitution procedure is only appropriate for a matrix that is upper triangular. Cases where a *pivot element* is nearly $0$ invite roundoff errors that can lead to incorrect solutions or a routine suggesting that no solution exists. We saw that we can use scaling to guide *row pivots* to make a coefficient matrix as close to diagonally dominant as possible, which will avoid the problems outlined here.

***

## Extras

The following is included as an FYI which may be helpful for some homework problems.

### Doolittle LU-decomposition with Scaled Row Pivoting

We can also adapt this Gaussian Elimination with Scaled Row Pivoting function to use the Doolittle LU-decomposition. As a reminder, using LU-decomposition is preferred when we would like to solve systems with the same coefficient matrix but multiple different constant vectors.

In [ ]:
def LUpivotDecomp(A, tol = 1.0e-12):
  n = A.shape[0]
  seq = np.array(range(n))

  s = scaleFactors(A)

  for k in range(0, n-1):
    p = np.argmax(np.abs(A[k:n, k])/s[k:n]) + k
    if abs(A[p, k]) < tol:
      print("Matrix is Singular")
      return None

    if p != k:
      swapRows(s, k, p)
      swapRows(A, k, p)
      swapRows(seq, k, p)

    #Elimination
    for i in range(k+1, n):
      if A[i, k] != 0.0:
        lam = A[i, k]/A[k, k]
        A[i, (k+1):n] = A[i, (k+1):n] - lam*A[k, (k+1):n]
        A[i, k] = lam

  return A, seq

def LUpivotSolve(A, b, seq):
  n = A.shape[0]

  #Rearrange constant vector due to Row Swaps in A
  x = b.copy()
  for i in range(n):
    x[i] = b[seq[i]]

  #Solution
  for k in range(1, n):
    x[k] = x[k] - np.dot(A[k, 0:k], x[0:k])

  x[n-1] = x[n-1]/A[n-1, n-1]
  for k in range(n-2, -1, -1):
    x[k] = (x[k] - np.dot(A[k, (k+1):n], x[(k+1):n]))/A[k, k]

  return x